In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf # tensolflow 1.x 버전을 사용한다.
tf.disable_v2_behavior() # tensorflow 2.x 버전의 문법을 사용하지 않겠다고 선언한다. => 1.x 버전을 사용할 시에 코딩

Instructions for updating:
non-resource variables are not supported in the long term


학습 데이터를 만든다.

In [2]:
xData = [1, 2, 3, 4, 5, 6, 7] # 1일 근무 시간
yData = [25000, 55000, 75000, 110000, 128000, 155000, 180000] # 근무시간에 따른 매출 금액, 실제값, 관측값, 답, 레이블

과적합을 방지하기 위해서 난수를 발생시켜 기울기(가중치)와 y절편(바이어스)을 정한다.  
과적합이란? 학습시킨 데이터에서는 높은 정확도를 보이지만, 학습시킨 데이터 이외의 데이터에서는 낮은 정확도를 보이는 문제점을 말한다.

In [3]:
# random_uniform([난수의 개수], 최소값, 최대값): 최대값과 최소값 사이에서 지정한 개수의 난수를 발생시킨다.
a = tf.Variable(tf.random_uniform([1], -100, 100)) # 기울기(가중치), -100 ~ 100 사이의 난수
b = tf.Variable(tf.random_uniform([1], -100, 100)) # y절편(바이어스), -100 ~ 100 사이의 난수
print('a = {}, b = {}'.format(a, b))

a = <tf.Variable 'Variable:0' shape=(1,) dtype=float32_ref>, b = <tf.Variable 'Variable_1:0' shape=(1,) dtype=float32_ref>


tensorflow 변수를 초기화하고 난수로 발생시킨 기울기와 y절편의 값을 확인한다.

In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a = {}, b = {}'.format(sess.run(a), sess.run(b)))

a = [39.520737], b = [-25.666832]


근무 시간과 매출 금액을 기억할 tensorflow에서 사용할 자료형(placeholder)를 선언한다.

In [7]:
x = tf.placeholder(dtype=tf.float32) # 근무 시간을 기억할 placeholder를 선언한다.
y = tf.placeholder(dtype=tf.float32) # 매출 금액을 기억할 placeholder를 선언한다.

1차 방정식 형태의 가설을 세우고 오차(손실, 비용) 함수를 정의한다.

In [8]:
# 1차 방정식 형태의 가설, Y => 예측값
Y = a * x + b
# 평균 제곱법을 이용하는 오차 함수
# 실제값(yData => y라는 placeholder에 저장한다.)과 예측값(Y)의 편차에 대한 제곱의 평균을 이용한다.
# square(): tensorflow에서 제곱값을 계산한다.
# reduce_mean(): tensorflow에서 평균을 계산한다.
cost = tf.reduce_mean(tf.square(Y - y))

경사 하강법 메소드로 오차 함수의 실행 결과를 가장 작게(최소) 만드는 방향으로 학습하도록 정의한다.

In [9]:
# 경사 하상 알고리즘의 학습율(learning rate)을 설정한다.
learning_rate = tf.Variable(0.01)
# train.GradientDescentOptimizer(학습율): 학습율에 따른 오차를 계산한다.
# minimize(오차 함수): 경하 하강 알고리즘에 따라 인수로 지정된 오차 함수 실행 결과의 최소값을 찾는다.
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

학습시킨다.

In [20]:
# Session을 만들고 변수를 초기화 시킨다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(5001):
    # 오차 함수의 결과를 가장 작게 만드는 학습을 할 수 있도록 placeholder에 데이터를 대입하고, 학습시킨다.
    # 학습할 데이터는 반드시 placeholder의 feed_dict 속성에 딕셔너리 타입으로 대입시켜야 한다.
    # 딕셔너리의 key에는 placeholder, value에는 placeholder에 저장할 리스트 또는 튜플 타입의 데이터를 지정한다.
    sess.run(train, feed_dict={x: xData, y: yData})
    
    # 일정 에포크(epoch) 단위마다 중간 결과를 출력한다.
    if i % 500 == 0:
        # epoch(i), 오차(cost), 기울기(a), y절편(b)
        #print('epoch: {:4d}, 오차: {:14.2f}, 기울기: {:9.3f}, y절편: {:9.3f}'.format(i, 
        #            sess.run(cost, feed_dict={x: xData, y: yData}), sess.run(a)[0], sess.run(b)[0]))
        print('epoch: %4d, 오차: %14.2f, 기울기: %9.3f, y절편: %9.3f' % (i, 
                    sess.run(cost, feed_dict={x: xData, y: yData}), sess.run(a), sess.run(b)))

print('기울기: {:9.3f}, y절편: {:9.3f}'.format(sess.run(a)[0], sess.run(b)[0]))
print('8시간 근무했을 때 예상 매출 금액: {:,.0f}원'.format(sess.run(a)[0] * 8 + sess.run(b)[0]))

epoch:    0, 오차:  4557753344.00, 기울기: 10421.152, y절편:  2129.685
epoch:  500, 오차:     8688544.00, 기울기: 25536.123, y절편:  1957.111
epoch: 1000, 오차:     8633839.00, 기울기: 25627.305, y절편:  1505.591
epoch: 1500, 오차:     8632673.00, 기울기: 25640.590, y절편:  1439.799
epoch: 2000, 오차:     8632647.00, 기울기: 25642.527, y절편:  1430.208
epoch: 2500, 오차:     8632652.00, 기울기: 25642.805, y절편:  1428.823
epoch: 3000, 오차:     8632657.00, 기울기: 25642.844, y절편:  1428.634
epoch: 3500, 오차:     8632656.00, 기울기: 25642.848, y절편:  1428.613
epoch: 4000, 오차:     8632656.00, 기울기: 25642.848, y절편:  1428.613
epoch: 4500, 오차:     8632656.00, 기울기: 25642.848, y절편:  1428.613
epoch: 5000, 오차:     8632656.00, 기울기: 25642.848, y절편:  1428.613
기울기: 25642.848, y절편:  1428.613
8시간 근무했을 때 예상 매출 금액: 206,571원


학습이 완료된 후 원하는 입력에 대한 예측 결과를 출력한다.

In [18]:
for i in range(8, 25):
    #print('{:2d}시간 근무했을 때 예상 매출 금액: {:,.0f}원'.format(i, sess.run(a)[0] * i + sess.run(b)[0]))
    print('{:2d}시간 근무했을 때 예상 매출 금액: {:,.0f}원'.format(i, sess.run(Y, feed_dict={x: [i]})[0]))

 8시간 근무했을 때 예상 매출 금액: 206,571원
 9시간 근무했을 때 예상 매출 금액: 232,214원
10시간 근무했을 때 예상 매출 금액: 257,857원
11시간 근무했을 때 예상 매출 금액: 283,500원
12시간 근무했을 때 예상 매출 금액: 309,143원
13시간 근무했을 때 예상 매출 금액: 334,786원
14시간 근무했을 때 예상 매출 금액: 360,428원
15시간 근무했을 때 예상 매출 금액: 386,071원
16시간 근무했을 때 예상 매출 금액: 411,714원
17시간 근무했을 때 예상 매출 금액: 437,357원
18시간 근무했을 때 예상 매출 금액: 463,000원
19시간 근무했을 때 예상 매출 금액: 488,643원
20시간 근무했을 때 예상 매출 금액: 514,286원
21시간 근무했을 때 예상 매출 금액: 539,928원
22시간 근무했을 때 예상 매출 금액: 565,571원
23시간 근무했을 때 예상 매출 금액: 591,214원
24시간 근무했을 때 예상 매출 금액: 616,857원
